In [9]:
import os
import re
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [10]:
# File credentials OAuth yang kamu download
CLIENT_SECRETS_FILE = "client_secret_1088100787674-kdeuavsdvjhpa23a2m2rvr7aukaq5l5o.apps.googleusercontent.com.json"

# Scopes yang dibutuhkan untuk mengakses dan moderasi komentar
SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [11]:
# Daftar kata kunci spam
SPAM_KEYWORDS = [
    "judi", "slot", "gacor", "link gacor", "togel", "bo slot", "agen slot", 
    "deposit pulsa", "slot online", "slot pakai dana", "toto macau"
]

In [12]:
def is_spam(comment_text):
    for keyword in SPAM_KEYWORDS:
        if re.search(rf'\b{keyword}\b', comment_text, re.IGNORECASE):
            return True
    return False

def main():
    # Autentikasi via browser
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        CLIENT_SECRETS_FILE, SCOPES)
    # credentials = flow.run_console()
    credentials = flow.run_local_server(port=0)


    # Build YouTube API client
    youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

    # Ganti dengan ID video YouTube kamu
    VIDEO_ID = "lGnUBl3pCF8"

    # Ambil komentar dari video
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=VIDEO_ID,
        maxResults=100,
        textFormat="plainText"
    )
    response = request.execute()

    for item in response["items"]:
        comment = item["snippet"]["topLevelComment"]
        comment_id = comment["id"]
        text = comment["snippet"]["textDisplay"]

        if is_spam(text):
            print(f"🚫 Komentar spam terdeteksi: {text}")
            # Set status komentar jadi "rejected"
            youtube.comments().setModerationStatus(
                id=comment_id,
                moderationStatus="rejected"
            ).execute()
            print("✅ Komentar berhasil dihapus")

if __name__ == "__main__":
    main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1088100787674-kdeuavsdvjhpa23a2m2rvr7aukaq5l5o.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60847%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=LLnrs64WSPaooET9ivijFJdu0i4inI&access_type=offline
🚫 Komentar spam terdeteksi: ayo join slot gacor pakai dana!
✅ Komentar berhasil dihapus
